In [1]:
using Pkg
Pkg.activate("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\env\\integrate\\")
using StaticArrays, ForwardDiff, BenchmarkTools, DifferentialEquations, DynamicalSystems

  Activating project at `C:\Users\Alex\Desktop\repo\2612\0903\dynamical-systems\env\integrate`


In [2]:
@inbounds U(y, p) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
@inbounds σ(x, p) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
@inbounds g(E, x, y, p, U_) = log( 1.0 + exp( (p[5] * U_ * x * E + p[11]  ) / (p[1]) ) )

@inbounds function TM(u, p, t)
    
    U_ = U(u[3], p)
    
    du1 = (-u[1] + p[1] * g(u[1], u[2], u[3], p, U_) ) / p[2]
    du2 = (1.0 - u[2]) / p[3] - U_*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2], p)
    
    return SVector(du1, du2, du3)
end

TM (generic function with 1 method)

@inbounds U(y, p, exp50) = p[8] + p[9] / ( 1.0 + exp50 )
@inbounds U_y(y, p, exp50) = (50.0 * p[9] * exp50) / (1.0 + exp50)^2
@inbounds g(E, x, y, p, U_) = exp((p[5]  * U_ * x * E + p[11]) / p[1])
@inbounds σ_der(x, p) = exp( (-20.0) * (x - p[6]) )
@inbounds function jacob_TM_(u, p, t)
    
    exp50 = exp(-50.0 * (u[3] - p[7]))
    
    U_ = U(u[3], p, exp50)
    Uy = U_y(u[3], p, exp50)
    g_ = g(u[1], u[2], u[3], p, U_)
    σ_deri = σ_der(u[2], p)
    
    g_plus = 1.0 + g_
    g_mult = g_ * U_
    g_plus_mult = p[2] * (g_plus)
    u1p5 = p[5] * u[1]
    Uyu2 = Uy * u[2]
    
    E_E = (-1.0 + ((J * u[2] * g_mult)) / (g_plus) ) / p[2]
    E_x = (u1p5 * g_mult) / (g_plus_mult)
    E_y = (u1p5 * Uyu2 * g_) / (g_plus_mult)
    
    x_E = -U_ * u[2]
    x_x = -1.0 / p[3] - U_ * u[1]
    x_y = -Uyu2 * u[1]
    
    y_x = 20.0 * p[10] * σ_deri / (1.0 + σ_deri)^2
    y_y = -1.0/p[4]
    
    SMatrix{3,3}(E_E, x_E, 0.0,
        E_x, x_x, y_x,
        E_y, x_y, y_y)
end

In [3]:
@inbounds function jacob_TM_(u, p, t)
    
    U(y, p, exp50) = p[8] + p[9] / ( 1.0 + exp50 )
    U_y(y, p, exp50) = (50.0 * p[9] * exp50) / (1.0 + exp50)^2
    g(E, x, y, p, U_) = exp((p[5]  * U_ * x * E + p[11]) / p[1])
    σ_der(x, p) = exp( (-20.0) * (x - p[6]) )
    exp50 = exp(-50.0 * (u[3] - p[7]))
    
    U_ = U(u[3], p, exp50)
    Uy = U_y(u[3], p, exp50)
    g_ = g(u[1], u[2], u[3], p, U_)
    σ_deri = σ_der(u[2], p)
    
    g_plus = 1.0 + g_
    g_mult = g_ * U_
    g_plus_mult = p[2] * (g_plus)
    u1p5 = p[5] * u[1]
    Uyu2 = Uy * u[2]
    
    E_E = (-1.0 + ((J * u[2] * g_mult)) / (g_plus) ) / p[2]
    E_x = (u1p5 * g_mult) / (g_plus_mult)
    E_y = (u1p5 * Uyu2 * g_) / (g_plus_mult)
    
    x_E = -U_ * u[2]
    x_x = -1.0 / p[3] - U_ * u[1]
    x_y = -Uyu2 * u[1]
    
    y_x = 20.0 * p[10] * σ_deri / (1.0 + σ_deri)^2
    y_y = -1.0/p[4]
    
    SMatrix{3,3}(E_E, x_E, 0.0,
        E_x, x_x, y_x,
        E_y, x_y, y_y)
end;

In [4]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const ΔU0 = 0.305;
U0 = 0.3; I0 = -1.61

-1.61

In [5]:
time = 1000
transient_time = 250
tstep = 0.001
integ_set = (alg = RK4(), adaptive = false, dt = tstep)

p = [α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [1.54672,  0.930353,  0.963849]
t = 0;

In [6]:
Jacobi(u0) = ForwardDiff.jacobian((x) -> TM(x, p, t), u0)

Jacobi (generic function with 1 method)

In [7]:
@benchmark  jacob_TM_(u0, p, t)

BenchmarkTools.Trial: 10000 samples with 975 evaluations.
 Range (min … max):   70.974 ns …  63.100 μs  ┊ GC (min … max):  0.00% … 99.19%
 Time  (median):      88.000 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   104.852 ns ± 712.662 ns  ┊ GC (mean ± σ):  10.43% ±  1.72%

  ▇▂▁ ▁██▄▃▂▂▂▂▁▁▂▁         ▁                                   ▂
  ██████████████████▇▇▇▇▇██████▇▇▆▅▅▅▆▅▅▅▅▅▄▄▆▄▄▅▄▄▅▄▅▄▄▆▅▄▅▃▄▂ █
  71 ns         Histogram: log(frequency) by time        244 ns <

 Memory estimate: 80 bytes, allocs estimate: 1.

In [8]:
@benchmark Jacobi(u0)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.450 μs …   5.990 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.590 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.715 μs ± 352.805 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅█▆▅▇▇▄▅▅▅▅▄▅▄▃▂▂▂ ▁                        ▁ ▁▁▂▁          ▂
  ██████████████████████▆▇▆▇▅▅▆▅▅▅▆▅▄▅▄▅▄▅▄▅▇█████████████▇██ █
  1.45 μs      Histogram: log(frequency) by time      2.96 μs <

 Memory estimate: 720 bytes, allocs estimate: 7.

In [9]:
ds = CoupledODEs(TM,  u0, p, diffeq = integ_set)
tan_ = TangentDynamicalSystem(ds, J = jacob_TM_)
tan_auto  = TangentDynamicalSystem(ds)

3-dimensional TangentDynamicalSystem
 deterministic:     true
 discrete time:     false
 in-place:          false
 dynamic rule:      TM
 jacobian:          ForwardDiff
 deviation vectors: 3
 parameters:        [1.58, 0.013, 0.08, 3.3, 3.07, 0.75, 0.4, 0.3, 0.305, 0.3, -1.61]
 time:              0.0
 state:             [1.54672, 0.930353, 0.963849]


In [10]:
@benchmark lyapunovspectrum(tan_, time)

BenchmarkTools.Trial: 9 samples with 1 evaluation.
 Range (min … max):  555.056 ms … 668.738 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     559.381 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   578.549 ms ±  38.268 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ███            ▁            ▁                               ▁  
  ███▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  555 ms           Histogram: frequency by time          669 ms <

 Memory estimate: 3.06 KiB, allocs estimate: 48.

In [11]:
@benchmark lyapunovspectrum(tan_auto, time)

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  700.209 ms … 809.450 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     701.423 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   724.024 ms ±  40.350 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                              
  █▁▁▁▁▁▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  700 ms           Histogram: frequency by time          809 ms <

 Memory estimate: 3.06 KiB, allocs estimate: 48.